In [7]:
import pandas as pd
import yfinance as yf
import datetime as dt
import requests


#Mock Data
dict_of_dates = {'AAPL':[["2017-01-03", "2017-01-30","2017-01-31"], ["2018-01-03", "2018-01-30","2018-01-31"]]}

#12:10 Start Clock

In [4]:
'''GET UNIVERSE HERE... (from csv?)'''


#Get Universe...

#Non CSV
#Fill in later 

div_univ = pd.read_csv('dividend_stocks.csv')
universe = div_univ.Symbol.to_list()
universe
print(len(universe))

4566


In [23]:
    '''HELPER Functions'''
    SANDBOX = True
    

    b = 'https://cloud.iexapis.com/'
    v = 'stable/'
    pre_token = '?token='
    public = 'pk_9b48f8bea8e8462e8226b03eeda290ec'
    
    sb = 'https://sandbox.iexapis.com/' #Sandbox
    sb_public = 'Tpk_635c676a17c64c95a91c39bfa823897d' #Sandbox
    
    if SANDBOX:
        b = sb
        public = sb_public
     
    HIST_DATES_BY_SYMBOL = {}
        
    def getHistDivDates(symbol, last = 2):
        assert last in [1, 2, 5], 'ERROR @153 -- in getHistDivDates last must be 1yr, 2yr, 5yr...'  
        
        #OLD Advanced Dividends...
        #NEWPT = '&token='
        #endpoint = f'/time-series/advanced_dividends/{symbol}?last={last}'
        #divs = requests.get(f'{self.b}{self.v}{endpoint}{NEWPT}{self.public}')
        b = 'https://cloud.iexapis.com/'
        v = 'stable/'
        pre_token = '?token='
        public = 'pk_9b48f8bea8e8462e8226b03eeda290ec'
        
        #NEW Basic Dividends Version
        endpoint = f'/stock/{symbol}/dividends/{last}y'
        try:
            divs = requests.get(f'{b}{v}{endpoint}{pre_token}{public}')      
        
            divs_list = divs.json()
        except:
            print(f'No divs returned for {symbol}')
            return
        
        list_of_trios = []
        for div in divs_list:
            try:
                #final = [v for k,v in div.items() if k in ['announceDate','exDate','recordDate']] #AdvDivs
                final = [v for k, v in div.items() if k in ['declaredDate','exDate','recordDate']]
                final = sorted(final)                      #LINE ONLY REQUIRED IN BASIC 
                list_of_trios += [final]
            except:
                #Should maybe return out here too?
                continue
            
        HIST_DATES_BY_SYMBOL[symbol] = list_of_trios
        return list_of_trios
    
    
#Testing...
getHistDivDates('AAPL')
HIST_DATES_BY_SYMBOL

{'AAPL': [['2020-07-30', '2020-08-24', '2020-08-31'],
  ['2020-07-30', '2020-08-07', '2020-08-10'],
  ['2020-04-30', '2020-05-08', '2020-05-11'],
  ['2020-01-28', '2020-02-07', '2020-02-10'],
  ['2019-10-30', '2019-11-07', '2019-11-11'],
  ['2019-07-30', '2019-08-09', '2019-08-12'],
  ['2019-04-30', '2019-05-10', '2019-05-13'],
  ['2019-01-29', '2019-02-08', '2019-02-11'],
  ['2018-11-01', '2018-11-08', '2018-11-12']]}

In [11]:
'''Test...'''
test = ['AAPL','CAT']
for symbol in test:#universe:
    #list_of_date_trios = getHistDivDates(symbol) #Above ^^^ (Remove self ... OR PUT IN PROGRAM)
    #HIST_DIVS_BY_SYMBOL[symbol] = list_of_date_trios
    getHistDivDates(symbol)
    


In [19]:
print(len(universe))
#print(HIST_DATES_BY_SYMBOL)

print('Bingo!')
HIST_DATES_BY_SYMBOL

4566
Bingo!


{'AAPL': [['2020-07-30', '2020-08-24', '2020-08-31'],
  ['2020-07-30', '2020-08-07', '2020-08-10'],
  ['2020-04-30', '2020-05-08', '2020-05-11'],
  ['2020-01-28', '2020-02-07', '2020-02-10'],
  ['2019-10-30', '2019-11-07', '2019-11-11'],
  ['2019-07-30', '2019-08-09', '2019-08-12'],
  ['2019-04-30', '2019-05-10', '2019-05-13'],
  ['2019-01-29', '2019-02-08', '2019-02-11'],
  ['2018-11-01', '2018-11-08', '2018-11-12']],
 'CAT': [['2020-06-10', '2020-07-17', '2020-07-20'],
  ['2020-04-08', '2020-04-17', '2020-04-20'],
  ['2019-12-11', '2020-01-17', '2020-01-21'],
  ['2019-10-09', '2019-10-18', '2019-10-21'],
  ['2019-05-02', '2019-07-19', '2019-07-22'],
  ['2019-04-10', '2019-04-18', '2019-04-22'],
  ['2018-12-12', '2019-01-18', '2019-01-22'],
  ['2018-10-10', '2018-10-19', '2018-10-22']]}

In [ ]:
for symbol in universe:
    getHistDivDates(symbol)

In [26]:
#HIST_DIVS_BY_SYMBOL
print(len(HIST_DATES_BY_SYMBOL))

4210


In [30]:
remove_empty_list_vals = {k:v for k, v in HIST_DATES_BY_SYMBOL.items() if v != []}

In [35]:
print(len(remove_empty_list_vals))
'AAPL' in remove_empty_list_vals

2394


True

In [36]:
HIST_DIVS = remove_empty_list_vals
print(len(HIST_DIVS))

2394


In [39]:
#Get final DF -- of Symbol, and HIST DATES for each symbol

def df_from_nested_dict(nested_dict = dict_of_dates, col_names = ['AD','ED','RD']):
    '''
    ACCEPTS self.hist_dates_by_symbol[symbol] = list_of_trios (from IEXDiv)
    Call analyzeAllHist -- will fill in the hist_dates_by_symbol dict...
    
    
    '''
    symbols = []
    dates = []
    Ads, Eds, Rds = [], [], []
    for k,v in nested_dict.items():
        
        #dates += [i for i in v]
        symbols += [k for i in range(len(v))]

        #dates += [i for i in v]    #NEEDS to be expanded... CAN be done with APPLY?

        Ads += [i[0] for i in v]
        Eds += [i[1] for i in v]
        Rds += [i[2] for i in v]
        

        c1, c2, c3 = col_names
        div_df = pd.DataFrame({'Symbol':symbols,c1:Ads,c2:Eds,c3:Rds})
        
    return div_df

FINAL = df_from_nested_dict(HIST_DIVS)

In [ ]:
FINAL

In [41]:
#Save JUST DATES to csv -- so we don't need to run again.
FINAL.to_csv('historical_divs.csv')

In [46]:
    '''HELPER FUNCTION'''
    from dateutil.relativedelta import relativedelta
    
    def getTrioPrices(symbol, selected):
        #Takes a symbol, and tuple of 3 dates.
        #plus_one = dt.datetime.strptime(selected[2], '%Y-%m-%d') + dt.timedelta(days=1) #dt.strftime("%Y-%m-%d")
        #plus_one_str = plus_one.strftime('%Y-%m-%d')
        
        plus_mo = dt.datetime.strptime(selected[2], '%Y-%m-%d') + relativedelta(months = 1)

        plus_mo_str = plus_mo.strftime('%Y-%m-%d')
        df = yf.download(symbol, start=selected[0], end=plus_mo_str).Close 

        #pdr is 750ms -- much slower
        df = df[df.index.isin(selected)]
        
        #rel_pct_changes = df.pct_change().dropna().to_list()
        
        return df.to_list()
    
    import yfinance as yf
    
    
    def apply_prices_to_dates(row):
        sym = row.Symbol
        trio = row.AD, row.ED, row.RD
        new_row = getTrioPrices(sym, trio)
        if len(new_row) != 3:
            return [-1,-1,-1]
        return getTrioPrices(sym, trio)

In [48]:
len(FINAL)

22280

In [49]:
#EXPAND -- version...

'''APPLY the prices, once we HAVE the dates...'''
prices_df = FINAL.apply(apply_prices_to_dates, axis=1, result_type = 'expand')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [52]:
#MAYYY -- need to do a TRY EXCEPT block INSIDE the apply -- return empty TUPLE [0,0,0] for example...

prices_df.to_csv('prices_df.csv')

In [53]:
len(prices_df)

22280

In [54]:
FINAL[prices_df.columns] = prices_df

In [56]:
FINAL.rename(columns = {0:'AP',1:'EP',2:'RP'},inplace=True)

In [58]:
FINAL.to_csv('final_df.csv')

In [61]:
#FINAL
index_to_drop = FINAL[ FINAL['AP'] <= 0 ].index 
index_to_drop

Int64Index([   17,    43,    53,    70,    73,    74,    75,    76,    78,
              130,
            ...
            22233, 22237, 22238, 22254, 22255, 22256, 22257, 22258, 22259,
            22262],
           dtype='int64', length=1163)

In [62]:
FINAL.drop(index_to_drop,inplace=True)

In [65]:
len(FINAL)
FINAL.to_csv('final_clean_df.csv')